# Import package

In [ ]:
import os
import math
import time 
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib
from collections import Counter

# Global variable

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.device('/gpu:0')
print(device_lib.list_local_devices())
label_classes = ['森森', '牛排', '巧巧', '小白', '花捲', '黑胖', '乖狗', '橘子', '烏龜', '松鼠', '笨鳥']

num_of_class = len(label_classes)
batch_size = 32
target_size = (224, 224)

# Preprocess data

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    # rotation_range=10,
    # shear_range=0.1,
    zoom_range=0.1,
    # fill_mode='nearest'
)

# train
train_generator = train_datagen.flow_from_directory(
    directory="data/",
    target_size=target_size,
    color_mode="rgb",
    class_mode='categorical',
    classes=label_classes,
    batch_size=batch_size,
    shuffle=True,
    subset='training'
)

for i, j in Counter(train_generator.classes).items():
    print(f"{label_classes[i]} with {j:3d} images")
print()

# valid
valid_generator = train_datagen.flow_from_directory(
    directory="data/",
    target_size=target_size,
    color_mode="rgb",
    class_mode='categorical',
    classes=label_classes,
    batch_size=batch_size,
    shuffle=True,
    subset='validation'
)
for i, j in Counter(valid_generator.classes).items():
    print(f"{label_classes[i]} with {j:3d} images")
print()
    
# test
test_datagen = ImageDataGenerator(
    rescale=1./255,
)
test_generator = test_datagen.flow_from_directory(
    directory="test_data/",
    target_size=target_size,
    color_mode="rgb",
    class_mode='categorical',
    classes=label_classes,
)
for i, j in Counter(test_generator.classes).items():
    print(f"{label_classes[i]} with {j:3d} images")
    

In [ ]:
# for my_batch in train_generator:
#     images = my_batch[0]
#     labels = my_batch[1]
#     for i in range(3):
#         print(label_classes[labels[i].argmax()])
#         plt.imshow(images[i])
#         plt.show()
#     break

# Build model

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    include_top = False,
    input_shape = target_size + (3, ),
    weights = 'imagenet'
)

base_model.trainable = False

# base_model.summary()

# # fine tune
# for layer in base_model.layers:
#     if(layer.name in ['Conv_1', 'Conv_2']):
#         layer.trainable = True
#     else:
#         layer.trainable = False

model = models.Sequential([
    base_model,
    # layers.Conv2D(filters=128, kernel_size=3, activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Flatten(),
    layers.Dropout(0.5),
    # layers.Dense(units=64, activation='relu'),
    layers.Dense(num_of_class, activation='softmax')
])

model.summary()

# Start training

In [71]:
# Set model

my_callback = [tf.keras.callbacks.ModelCheckpoint(
    filepath='weights_save/weights_L_{epoch:03d}_{val_accuracy:.2f}.hdf5',
    # filepath='weights_save/weights_S_{epoch:03d}_{val_accuracy:.2f}.hdf5',
    monitor='val_accuracy',
    save_weights_only=True,
    mode='max'
)]

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=valid_generator,
    validation_steps=STEP_SIZE_VALID,
    epochs=10,
    # callbacks=my_callback
)

acc = history.history['accuracy']
epochs = range(1, len(acc) + 1)
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc', linewidth=0.5)
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.grid()
# 儲存acc學習曲線
# plt.savefig('./acc.png')
plt.show()
# 畫出loss學習曲線
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss', linewidth=0.5)
plt.title('Training and validation loss')
plt.legend(loc='upper right')
plt.grid()
# 儲存loss學習曲線
# plt.savefig('loss.png')
plt.show()

In [73]:
pred_train = model.evaluate(train_generator, verbose=0, steps=STEP_SIZE_TRAIN)
pred_val = model.evaluate(valid_generator, verbose=0, steps=STEP_SIZE_VALID)

print("Train loss:", pred_train[0])
print("Train accuracy:", pred_train[1])
print("Validation loss:", pred_val[0])
print("Validation accuracy:", pred_val[1])

Train loss: 0.08507218956947327
Train accuracy: 0.9817073345184326
Validation loss: 0.16960853338241577
Validation accuracy: 0.9453125


In [74]:
test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_generator = test_datagen.flow_from_directory(
    directory='test/',
    target_size=target_size,
    color_mode="rgb",
    class_mode='categorical',
    classes=label_classes,
)

images = []
labels = []
predictions = []

for data, label in test_generator:
    
    prediction = np.argmax(model.predict(data), axis=1)
    predictions.extend(prediction)
    
    images.extend(data)
    
    labels.extend(np.argmax(label, axis=1))
    
    if (len(predictions) == test_generator.n):
        break

Found 7 images belonging to 11 classes.


In [ ]:
images = []
labels = []
predictions = []

for data, label in valid_generator:
    
    prediction = np.argmax(model.predict(data), axis=1)
    predictions.extend(prediction)
    
    images.extend(data)
    
    labels.extend(np.argmax(label, axis=1))
    
    if (len(predictions) == valid_generator.n):
        break

In [ ]:
for i in range(9, 18):
    print(f"label={label_classes[labels[i]]} | predict={label_classes[predictions[i]]}")
    plt.imshow(images[i])
    plt.show()

# Convert to tflite

In [77]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("model.tflite", "wb") as fh:
    fh.write(tflite_model)

c:\Users\user\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpi58y9wfo\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpi58y9wfo\assets
